<blockquote>
    <h1>Exercise 5.7</h1>
    <p>In Sections 5.3.2 and 5.3.3, we saw that the <code>cv.glm()</code> function can be
used in order to compute the LOOCV test error estimate. Alterna-
tively, one could compute those quantities using just the <code>glm()</code> and
<code>predict.glm()</code> functions, and a for loop. You will now take this ap-
proach in order to compute the LOOCV error for a simple logistic
regression model on the <code>Weekly</code> data set. Recall that in the context
of classification problems, the LOOCV error is given in (5.4).</p>
    <ol>
        <li>Fit a logistic regression model that predicts $\mathrm{Direction}$ using $\mathrm{Lag1}$ and $\mathrm{Lag2}$.</li>
        <li>Fit a logistic regression model that predicts $\mathrm{Direction}$ using $\mathrm{Lag1}$ and $\mathrm{Lag2}$ <i>using all but the first observation</i>.</li>
        <li>Use the model from 2 to predict the direction of the first observation. You can do this by predicting that the first observation will go up if $P (\mathrm{Direction}="\mathrm{Up}"| \mathrm{Lag1}, \mathrm{Lag2}) > 0.5$. Was this observation correctly classified?</li>
        <li>
            Write a for loop from $i = 1$ to $i = n$, where $n$ is the number of observations in the data set, that performs each of the following steps:
            <ol>
                <li>Fit a logistic regression model using all but the $i$th observation to predict $\mathrm{Direction}$ using $\mathrm{Lag1}$ and $\mathrm{Lag2}$.</li>
                <li>Compute the posterior probability of the market moving up for the $i$th observation.</li>
                <li>Use the posterior probability for the ith observation in order to predict whether or not the market moves up.</li>
                <li>Determine whether or not an error was made in predicting the direction for the $i$th observation. If an error was made, then indicate this as a $1$, and otherwise indicate it as a $0$.</li>
            </ol>
        </li>
        <li>Take the average of the $n$ numbers obtained in 4 in order to obtain the LOOCV estimate for the test error. Comment on the results.</li>
    </ol>
</blockquote>

In [1]:
import pandas as pd
import numpy as np

# https://stackoverflow.com/questions/34398054/ipython-notebook-cell-multiple-outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import statsmodels.api as sm

<h3>Exercise 5.7.1</h3>
<blockquote>
    <i>Fit a logistic regression model that predicts $\mathrm{Direction}$ using $\mathrm{Lag1}$ and $\mathrm{Lag2}$.</i>
</blockquote>

In [2]:
df = pd.read_csv("../../DataSets/Weekly/Weekly.csv")
df = df.reindex(columns=['Year', 'Today', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume', 'Direction'])
df['Direction01'] = np.where(df['Direction'] == 'Up', 1, 0)
df.insert(0, 'Intercept', 1)
targetColumn = ['Direction01']
descriptiveColumns = ['Intercept', 'Lag1', 'Lag2']
df_X = df[descriptiveColumns]
df_Y = df[targetColumn]
model = sm.Logit(df_Y, df_X)
fitted = model.fit()
fitted.summary()

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Direction01   No. Observations:                 1089
Model:                          Logit   Df Residuals:                     1086
Method:                           MLE   Df Model:                            2
Date:                Sat, 18 Jan 2020   Pseudo R-squ.:                0.005335
Time:                        21:04:01   Log-Likelihood:                -744.11
converged:                       True   LL-Null:                       -748.10
Covariance Type:            nonrobust   LLR p-value:                   0.01848
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2212      0.061      3.599      0.000       0.101       0.342
Lag1          -0.0387      0.026     -1.477      0.140      -0.090       0.013
Lag2           0.0602      0.027      2.270      0.023       0.008       0.112
==============================================================================
"""

<h3>Exercise 5.7.2</h3>
<blockquote>
    <i>Fit a logistic regression model that predicts $\mathrm{Direction}$ using $\mathrm{Lag1}$ and $\mathrm{Lag2}$ <i>using all but the first observation</i>.</i>
</blockquote>

In [3]:
df_X_train = df[descriptiveColumns].iloc[1:]
df_Y_train = df[targetColumn].iloc[1:]
df_X_test = df[descriptiveColumns].iloc[0]
df_Y_test = df[targetColumn].iloc[0]
model = sm.Logit(df_Y, df_X)
fitted = model.fit()
fitted.summary()

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Direction01   No. Observations:                 1089
Model:                          Logit   Df Residuals:                     1086
Method:                           MLE   Df Model:                            2
Date:                Sat, 18 Jan 2020   Pseudo R-squ.:                0.005335
Time:                        21:04:01   Log-Likelihood:                -744.11
converged:                       True   LL-Null:                       -748.10
Covariance Type:            nonrobust   LLR p-value:                   0.01848
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2212      0.061      3.599      0.000       0.101       0.342
Lag1          -0.0387      0.026     -1.477      0.140      -0.090       0.013
Lag2           0.0602      0.027      2.270      0.023       0.008       0.112
==============================================================================
"""

<h3>Exercise 5.7.3</h3>
<blockquote>
    <i>Use the model from 2 to predict the direction of the first observation. You can do this by predicting that the first observation will go up if $P (\mathrm{Direction}="\mathrm{Up}"| \mathrm{Lag1}, \mathrm{Lag2}) > 0.5$. Was this observation correctly classified?</i>
</blockquote>

In [4]:
df_Y_test.iloc[0]
sr_Y_pred = fitted.predict(df_X_test.to_numpy())
sr_Y_pred[0]

0

0.570609187903177

<p>The model incorrectly classifies this observation as "Up".</p>

<h3>Exercise 5.7.4</h3>
<blockquote>
    <i>Write a for loop from $i = 1$ to $i = n$, where $n$ is the number of observations in the data set, that performs each of the following steps:
            <ol>
                <li>Fit a logistic regression model using all but the $i$th observation to predict $\mathrm{Direction}$ using $\mathrm{Lag1}$ and $\mathrm{Lag2}$.</li>
                <li>Compute the posterior probability of the market moving up for the $i$th observation.</li>
                <li>Use the posterior probability for the ith observation in order to predict whether or not the market moves up.</li>
                <li>Determine whether or not an error was made in predicting the direction for the $i$th observation. If an error was made, then indicate this as a $1$, and otherwise indicate it as a $0$.</li>
            </ol></i>
</blockquote>

In [5]:
n = df.shape[0]
total_correct = 0
for i in range(n):
    excluded = [i]    
    df_X_train = df[descriptiveColumns].drop(excluded, axis=0, inplace=False)
    df_Y_train = df[targetColumn].drop(excluded, axis=0, inplace=False)
    df_X_test = df[descriptiveColumns].iloc[i]
    df_Y_test = df[targetColumn].iloc[i]
    model = sm.Logit(df_Y, df_X)
    fitted = model.fit()
#     fitted.summary()

    sr_Y_pred = fitted.predict(df_X_test.to_numpy())
    assert sr_Y_pred.shape == (1, )
    Y_pred = 1 if sr_Y_pred[0] > 0.5 else 0
    if df_Y_test.iloc[0] != Y_pred:
        total_correct += 1
    

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

...

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683297
  

<h3>Exercise 5.7.5</h3>
<blockquote>
    <i>Take the average of the $n$ numbers obtained in 4 in order to obtain the LOOCV estimate for the test error. Comment on the results.</i>
</blockquote>

In [6]:
total_correct
(total_correct / n) * 100

484

44.44444444444444

<p>The model misclassified about $44 \%$ of the observations.</p>